In [3]:
import os
from decimal import Decimal
from datetime import datetime, time, timedelta
import pytz
import uuid
import enum

from dotenv import load_dotenv
import pandas as pd
from stock_indicators import indicators
from stock_indicators.indicators.common.quote import Quote

from tinkoff.invest.sandbox.client import SandboxClient
from tinkoff.invest import InstrumentIdType, InstrumentStatus, MoneyValue, CandleInterval, OrderDirection, OrderType
from tinkoff.invest.utils import decimal_to_quotation, quotation_to_decimal
# InstrumentExchangeType

In [4]:
load_dotenv()
t_invest_token = os.getenv('TINVEST_TOKEN')
account_name = 'SimpleMomentumBot1'

To create a jupyter_notebook_config.py file, with all the defaults commented out, you can use the following command line:

$ jupyter notebook --generate-config

Open the file and search for c.NotebookApp.iopub_data_rate_limit

Comment out the line c.NotebookApp.iopub_data_rate_limit = 1000000 and change it to a higher default rate. l used c.NotebookApp.iopub_data_rate_limit = 10000000

# Получение текущего состояния

In [5]:
def get_account(client):
    accounts = client.users.get_accounts()
    for account in accounts.accounts:
        if account.name == account_name:
            return account
    return None

In [6]:
with SandboxClient(t_invest_token) as client:
    
    print(client.users.get_info())
    
    print("------")
    print(client.users.get_accounts())
    
    print("------")
    print(client.users.get_margin_attributes(account_id=get_account(client).id) )
    
    print("------")
    #print(client.users.get_user_tariff() )

GetInfoResponse(prem_status=False, qual_status=False, qualified_for_work_with=['derivative', 'structured_bonds', 'closed_fund', 'bond', 'structured_income_bonds', 'russian_shares', 'leverage', 'foreign_shares', 'foreign_etf', 'foreign_bond', 'option'], tariff='sandbox', user_id='', risk_level_code='')
------
GetAccountsResponse(accounts=[Account(id='675e84ac-36da-4c28-a737-4c82f8e1f3ee', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='SimpleMomentumBot1', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2025, 5, 17, 9, 55, 2, 998807, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_FULL_ACCESS: 1>)])
------
GetMarginAttributesResponse(liquid_portfolio=MoneyValue(currency='rub', units=49999, nano=852260000), starting_margin=MoneyValue(currency='rub', units=0, nano=0), minimal_margin=MoneyValue(currency='rub', units=0, nano=0), funds_sufficiency_level=

# Добавление аккаунта в песочницу

In [7]:
#with SandboxClient(t_invest_token) as client:
#        client.sandbox.open_sandbox_account(name="SimpleMomentumBot1")

In [8]:
#with SandboxClient(t_invest_token) as client:
#    money = decimal_to_quotation(Decimal(50000))    
#    client.sandbox.sandbox_pay_in(account_id='675e84ac-36da-4c28-a737-4c82f8e1f3ee', 
#                                  amount=MoneyValue(units=money.units, nano=money.nano, currency='rub'))

# Получение данных по тикеру

In [9]:
#ticker = "GDM5"
ticker = "SBER"

In [10]:
data = []
with SandboxClient(t_invest_token) as client:
    #print(client.instruments.future_by(id="GDM5", id_type=InstrumentIdType.INSTRUMENT_ID_TYPE_TICKER, class_code="future"))
    fs = list(filter(lambda x: x.ticker == ticker,  client.instruments.shares().instruments))    
    if(len(fs) > 0):
        figi = fs[0].figi     
        for candle in client.get_all_candles(
            figi=figi,
            #from_=pytz.UTC.localize(datetime.combine(datetime.now(), time.min) - timedelta(days=1)),
            from_=pytz.UTC.localize(datetime(2025,3,1)),
            interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
        ):
            data += [
                [
                 candle.time,
                 float(quotation_to_decimal(candle.open)), 
                 float(quotation_to_decimal(candle.close)),
                 float(quotation_to_decimal(candle.high)),
                 float(quotation_to_decimal(candle.low)),
                 candle.volume
                ]]

ticker_data_draft = pd.DataFrame(data, columns=['UTC', 'open', 'close','high','low','volume']) 

In [11]:
ticker_data_draft

,UTC,open,close,high,low,volume
0,2025-03-01 00:00:00+00:00,309.60,309.68,309.68,309.60,18
1,2025-03-01 00:01:00+00:00,309.60,309.60,309.60,309.60,6
2,2025-03-01 00:08:00+00:00,309.68,309.68,309.68,309.68,3
3,2025-03-01 00:09:00+00:00,309.60,309.68,309.68,309.60,4
4,2025-03-01 00:10:00+00:00,309.68,309.68,309.68,309.68,12
...,...,...,...,...,...,...
86918,2025-05-25 18:06:00+00:00,302.00,302.00,302.00,302.00,3
86919,2025-05-25 18:07:00+00:00,302.17,302.17,302.17,302.17,1
86920,2025-05-25 18:08:00+00:00,302.00,301.99,302.00,301.99,50
86921,2025-05-25 18:13:00+00:00,302.17,302.17,302.17,302.17,10


In [12]:
ticker_data_draft.to_csv("data\\2025-"+ticker+"\\2025-"+ticker+".csv", sep=';', encoding='utf-8', header=False)

In [13]:
ticker_data = ticker_data_draft.copy()
ticker_data.index = pd.DatetimeIndex(ticker_data_draft['UTC'])
ticker_data.drop(['UTC'], axis=1, inplace=True)

In [14]:
ticker_data

,open,close,high,low,volume
UTC,,,,,
2025-03-01 00:00:00+00:00,309.60,309.68,309.68,309.60,18
2025-03-01 00:01:00+00:00,309.60,309.60,309.60,309.60,6
2025-03-01 00:08:00+00:00,309.68,309.68,309.68,309.68,3
2025-03-01 00:09:00+00:00,309.60,309.68,309.68,309.60,4
2025-03-01 00:10:00+00:00,309.68,309.68,309.68,309.68,12
...,...,...,...,...,...
2025-05-25 18:06:00+00:00,302.00,302.00,302.00,302.00,3
2025-05-25 18:07:00+00:00,302.17,302.17,302.17,302.17,1
2025-05-25 18:08:00+00:00,302.00,301.99,302.00,301.99,50


# Размещение заявок

In [37]:
figi = "BBG004730N88"  # BBG004730ZJ9 - VTBR / BBG004730N88 - SBER

In [38]:
with SandboxClient(t_invest_token) as client:
    statuses = client.market_data.get_trading_statuses(instrument_ids=[figi])
    client.market_data.get_trading_status(figi=
    print(statuses)

GetTradingStatusesResponse(trading_statuses=[GetTradingStatusResponse(figi='BBG004730N88', trading_status=<SecurityTradingStatus.SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING: 1>, limit_order_available_flag=False, market_order_available_flag=False, api_trade_available_flag=True, instrument_uid='e6123145-9665-43e0-8413-cd61b8aa9b13', bestprice_order_available_flag=False, only_best_price=False)])


In [40]:
with SandboxClient(t_invest_token) as client:
    account = get_account(client)
    
    order_id = str(uuid.uuid4())
    post_order_response = client.orders.post_order(
            quantity=1,
            direction=OrderDirection.ORDER_DIRECTION_BUY,
            account_id=account.id,
            order_type=OrderType.ORDER_TYPE_MARKET,
            order_id=order_id,
            instrument_id=figi,
        )

    post_order_response

bf5b237d957aac4552798893215ad5a9 PostOrder INVALID_ARGUMENT 30079


RequestError: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30079', Metadata(tracking_id='bf5b237d957aac4552798893215ad5a9', ratelimit_limit='200, 200;w=60', ratelimit_remaining=198, ratelimit_reset=58, message='Instrument is not available for trading'))

In [34]:
post_order_response

PostOrderResponse(order_id='18eecacf-2e82-4784-bf17-7680f9847b4c', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=95, nano=480000000), executed_order_price=MoneyValue(currency='rub', units=95, nano=480000000), total_order_amount=MoneyValue(currency='rub', units=95, nano=480000000), initial_commission=MoneyValue(currency='rub', units=0, nano=47740000), executed_commission=MoneyValue(currency='rub', units=0, nano=47740000), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004730ZJ9', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=95, nano=480000000), order_type=<OrderType.ORDER_TYPE_MARKET: 2>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='8e2b0325-0292-4654-8a18-4f63ed3b0e09', order_request_id='53b430d1-2c50-47f6-9023-e7f55b3fa51f', response_metadata=Response

In [41]:
with SandboxClient(t_invest_token) as client:
        response = client.users.get_accounts()
        accounts = [account.id for account in response.accounts]
        for response in client.operations.get_positions .positions_stream(accounts=accounts):
            print(response)

PositionsStreamResponse(subscriptions=PositionsSubscriptionResult(accounts=[PositionsSubscriptionStatus(account_id='675e84ac-36da-4c28-a737-4c82f8e1f3ee', subscription_status=<PositionsAccountSubscriptionStatus.POSITIONS_SUBSCRIPTION_STATUS_SUCCESS: 1>)], tracking_id='', stream_id=''), position=None, ping=None, initial_positions=None)


KeyboardInterrupt: 

In [47]:
with SandboxClient(t_invest_token) as client:
        print(client.operations.get_portfolio(account_id=get_account(client).id).positions)

[PortfolioPosition(figi='RUB000UTSTOM', instrument_type='currency', quantity=Quotation(units=49904, nano=472260000), average_position_price=MoneyValue(currency='', units=0, nano=0), expected_yield=Quotation(units=0, nano=0), current_nkd=MoneyValue(currency='', units=0, nano=0), average_position_price_pt=Quotation(units=0, nano=0), current_price=MoneyValue(currency='', units=0, nano=0), average_position_price_fifo=MoneyValue(currency='', units=0, nano=0), quantity_lots=Quotation(units=49904, nano=472260000), blocked=False, blocked_lots=Quotation(units=0, nano=0), position_uid='33e24a92-aab0-409c-88b8-f2d57415b920', instrument_uid='a92e2e25-a698-45cc-a781-167cf465257c', var_margin=MoneyValue(currency='', units=0, nano=0), expected_yield_fifo=Quotation(units=0, nano=0), daily_yield=MoneyValue(currency='', units=0, nano=0), ticker=''), PortfolioPosition(figi='BBG004730ZJ9', instrument_type='share', quantity=Quotation(units=1, nano=0), average_position_price=MoneyValue(currency='rub', units

# Торговый бот

In [51]:
class StrategyCommand(enum.Enum):
    BEAR_GO = -2
    BEAR_HOLD = -1
    CLOSE = 0
    BULL_HOLD = 1
    BULL_GO = 2

In [50]:
class Strategy:
    
    # конфигурация
    frame_size = 22
    #frame_size = 120
    fractal_in_enabled  = False
    fractal_out_enabled = False
    fractal_size = 4
    
    def __init__(self):
        
        return

    # Расчет индикаторов
    def calc_indicators(self, data ):
        quotes = [Quote(d,o,h,l,c,v) for d,o,h,l,c,v in zip(data.index, data['open'], data['high'], data['low'], data['close'], data['volume'])]
        alligator = indicators.get_alligator(quotes)
        fractal = indicators.get_fractal(quotes, self.fractal_size)
        result = data.copy()
        result["jaw"] = [ a.jaw for a in alligator]
        result["teeth"] = [ a.teeth for a in alligator]
        result["lips"] = [ a.lips for a in alligator]
        result["fractal_bear"] = [ f.fractal_bear if f.fractal_bear != None else np.nan for f in fractal]
        result["fractal_bull"] = [ f.fractal_bull if f.fractal_bull != None else np.nan for f in fractal]
        
        return result

    # helper
    def get_params(self,data):
        
        index = data.tail(1).index[0]
        prev_index = data.tail(2).index[0]
        fractal_index = data.tail(self.fractal_size+1).index[0]
        return (
                index,
                data.at[index, 'open'], 
                data.at[index, 'close'],
                data.at[index, 'high'],
                data.at[index, 'low'],
                data.at[index, 'volume'],

                data.at[index, 'jaw'  ] if data.at[index, 'jaw'  ] != None else np.nan,
                data.at[index, 'teeth'] if data.at[index, 'teeth'] != None else np.nan,
                data.at[index, 'lips' ] if data.at[index, 'lips' ] != None else np.nan,

                data.at[prev_index, 'jaw'  ] if data.at[prev_index, 'jaw'  ] != None else np.nan,
                data.at[prev_index, 'teeth'] if data.at[prev_index, 'teeth'] != None else np.nan,
                data.at[prev_index, 'lips' ] if data.at[prev_index, 'lips' ] != None else np.nan,

                data.at[fractal_index, 'fractal_bear'],
                data.at[fractal_index, 'fractal_bull']
           )
    
    # обработка шага стратегии
    def next(self, data):

        
        # инициализация локальных переменных
        indicators = self.calc_indicators(data)
        (
            time,   open_,  close,   high,   low,   volume,   
            jaw,   teeth,   lips,   
            prev_jaw,   prev_teeth,   prev_lips,
            fractal_bear,   fractal_bull
        )   = self.get_params(indicators)

        command = StrategyCommand.CLOSE
        stop_loss = lips
        last_row = indicators.tail(1).copy()
        last_row.loc[ last_row.index[0], 'fractal_bull'] = fractal_bull 
        last_row.loc[ last_row.index[0], 'fractal_bear'] = fractal_bear
        
        # прерываем исполнение, если недостаточно данных
        if ( math.isnan(jaw) or 
             math.isnan(teeth) or 
             math.isnan(lips) or
             volume == 0 
            #or len(indicators.index) < self.frame_size
           ):
            return command, stop_loss, last_row
        
        #Определяем значение команды 
        # проверки на быка
        # проверяем главное условие аллигатора
        if   ( close > lips and lips > teeth and lips > jaw):
            command = StrategyCommand.BULL_HOLD
            
                
            if(
                # проверяем, что условие наступило только на текущем баре, а не сохраняется уже давно
                (lips >= low  or not (prev_lips > prev_teeth and prev_lips > prev_jaw) or math.isnan(prev_jaw))
                
                # проверяем доп. условие входа по пробитию уровня, чтобы отсечь мелкие тренды
                and (not self.fractal_in_enabled or indicators['fractal_bear'].sum() > 0 and np.nanmin(indicators['fractal_bear']) < close)
            ):    
                command = StrategyCommand.BULL_GO

        #проверки на медведя
        # проверяем главное условие аллигатора
        elif ( close < lips and lips < teeth and lips < jaw ):
            command = StrategyCommand.BEAR_HOLD
            
                
            if (
                # проверяем, что условие наступило только на текущем баре, а не сохраняется уже давно
                (lips <= high or not (prev_lips < prev_teeth and prev_lips < prev_jaw) or math.isnan(prev_jaw))
                
                # проверяем доп. условие входа по пробитию уровня, чтобы отсечь мелкие тренды
                and (not self.fractal_in_enabled or indicators['fractal_bull'].sum() > 0 and np.nanmax(indicators['fractal_bull']) > close)
            ):
                command = StrategyCommand.BEAR_GO
            
        # проверка на выход из позиции по фракталу
        if (
            #прерывание быка по медвежему фракталу
            (
                (command == StrategyCommand.BULL_GO or command == StrategyCommand.BULL_HOLD)
                and not math.isnan(fractal_bear) and self.fractal_out_enabled
            )
            #прерыване медведя по бычьему фракталу
            or (
                (command == StrategyCommand.BEAR_GO or command == StrategyCommand.BEAR_HOLD)
                and not math.isnan(fractal_bull) and self.fractal_out_enabled
            )   
        ):
            command = StrategyCommand.CLOSE

        return command, stop_loss, last_row